In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
#Đọc file 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Giải nén dữ liệu 
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
#Giải nén dữ liệu 
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
#Dùng pd để đọc 
train_data = pd.read_csv('train.tsv', sep='\t')
train_data.head(5)

In [ ]:
#đếm tổng các nhãn có giá trị null của tập train. Nhận thấy có rất nhiều giá trị rỗng của nhãn brand_name và một số ít ở category_name
train_data.isnull().sum()

In [ ]:
#kiểm tra số chiều của ma trận và các nhãn
print(train_data.shape)
print(train_data.columns)

In [ ]:
#lọc các sản phẩm có giá > 0 
train_data = train_data[train_data['price'] > 0].reset_index(drop=True)
train_data,validation_data=train_test_split(train_data,test_size=0.1,random_state=42)
print(train_data.shape)
print(validation_data.shape)

In [ ]:
# Kiểm tra các phần tử null của tập train
train_data.isnull().sum()

In [ ]:
# Kiểm tra các phần tử null của tập validation
validation_data.isnull().sum()

In [ ]:
train = train_data.copy()
valid = validation_data.copy()

In [ ]:
#chia category_name thành các sub_category để phân chia thông tin rõ ràng hơn, 
def split_categories(category):
    try:
      sub_category1,sub_category2,sub_category3 = category.split("/")
      return sub_category1,sub_category2,sub_category3
    except:
      return ("No label","No label","No label")

def create_split_categories(data):
    data['sub_category1'],data['sub_category2'],data['sub_category3']=zip(*data['category_name'].\
                                                                  apply(lambda x: split_categories(x)))

In [ ]:
create_split_categories(train_data)
create_split_categories(validation_data)

In [ ]:
#thay thế các giá trị null bằng các giá trị unknow_cat, unknow_brand và unknow_description
def fill_missing_values(data):
    data['category_name'].fillna('unknown_cat', inplace=True)
    data['brand_name'].fillna('unknown_brand', inplace=True)
    data['item_description'].fillna('unknown_description', inplace=True)
    return data

In [ ]:
fill_missing_values(train_data)
fill_missing_values(validation_data)

In [ ]:
test_data = pd.read_csv('test_stg2.tsv',sep='\t')
test = test_data.copy()

In [ ]:
#5 hàng dữ liệu đầu tiên của tệp test
test_data.head(5)

In [ ]:
#tệp test gồm 3460725 hàng và 7 cột
test_data.shape

In [ ]:
#Tệp test có giá trị null ở hàng brand_name là lớn nhất, tiếp theo đó là category_name.
test_data.isnull().sum()

In [ ]:
#tách categories cho têp test và thay thế các giá trị NaN thành unknow...
create_split_categories(test_data)
fill_missing_values(test_data)

In [ ]:
#tính logarit giá sản phẩm của tập train để cố gắng đưa về phân phối chuẩn
train_data['log_prices']= np.log(train_data['price']+1)

In [ ]:
#tính logarit giá sản phẩm của tập valid để cố gắng đưa về phân phối chuẩn
validation_data['log_prices']= np.log(validation_data['price']+1)

In [ ]:
#vẽ đồ thị biểu diễn giá trị logarit của giá
sns.kdeplot(data=train_data['log_prices'])
plt.title("Distribution of Price variable")
plt.grid(color='b')

In [ ]:
train_data.head(5)

#**Text processing**

In [ ]:
#import thư viện NLTK để xóa stopword
import nltk
nltk.download('stopwords')

In [ ]:
#xóa stop words (a, an, the,...) 
from nltk.corpus import stopwords

stop = stopwords.words('english')

def remove_stop_words(x):
    x = ' '.join([i for i in x.lower().split(' ') if i not in stop])
    return x

In [ ]:
#xóa stopwords trong các tệp train, valid và test
train_data['item_description'] = train_data['item_description'].apply(remove_stop_words)
validation_data['item_description'] = validation_data['item_description'].apply(remove_stop_words)
test_data['item_description'] = test_data['item_description'].apply(remove_stop_words)

In [ ]:
#Tiền xử lý, chuyển những ký hiệu viết tắt thành các từ hoàn chỉnh, và chỉnh toàn bộ các chữ in hoa thành in thường. thay thế các ký tự \r, \n, \\ 
#thành các ký tự rỗng
from tqdm import tqdm
import re

def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def text_preprocessing(text_col):
  preprocessed_total = []
  for sentence in tqdm(text_col.values):
    sent = decontracted(sentence)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    preprocessed_total.append(sent.lower().strip())
  return preprocessed_total

In [ ]:
#áp dụng các hàm tiền xử lý đã viết ở bên trên cho các tệp train, validation, test cho các trường description và name.
train_data['item_description']=text_preprocessing(train_data['item_description'])
validation_data['item_description']=text_preprocessing(validation_data['item_description'])
test_data['item_description']=text_preprocessing(test_data['item_description'])

train_data['name']=text_preprocessing(train_data['name'])
validation_data['name']=text_preprocessing(validation_data['name'])
test_data['name']=text_preprocessing(test_data['name'])

In [ ]:
#so sánh giữa tập train_data sau khi tiền xử lý và tập train chưa được xử lý. Ta có thể thấy, những ký tự đặc biệt đã được xóa, ký thực viết hoa sẽ thành 
# viết thường.
print(train_data['item_description'].iloc[33],len(train_data['item_description'].iloc[33].split(' ')))
print(train['item_description'].iloc[33],len(train['item_description'].iloc[33].split(' ')))

In [ ]:
#hàm xử lý dữ liệu
def clean_cat(cat_col):
    cat_list = []
    for i in tqdm(cat_col.values):
        i = re.sub('[^A-Za-z0-9]+', ' ', i)
        i = i.replace(' ','')
        i = i.replace('&','_')
        cat_list.append(i.strip())
    
    return cat_list

In [ ]:
#sử dụng hàm xử lý dữ liệu bên trên để làm sạch dữ liệu cho categories cho 3 tệp train_data, validation_data và test_data
train_data['sub_category1'] = clean_cat(train_data['sub_category1'])
validation_data['sub_category1'] = clean_cat(validation_data['sub_category1'])
test_data['sub_category1'] = clean_cat(test_data['sub_category1'])

train_data['sub_category2'] = clean_cat(train_data['sub_category2'])
validation_data['sub_category2'] = clean_cat(validation_data['sub_category2'])
test_data['sub_category2'] = clean_cat(test_data['sub_category2'])

train_data['sub_category3'] = clean_cat(train_data['sub_category3'])
validation_data['sub_category3'] = clean_cat(validation_data['sub_category3'])
test_data['sub_category3'] = clean_cat(test_data['sub_category3'])

In [ ]:
#sử dụng hàm xử lý dữ liệu bên trên để làm sạch dữ liệu cho brand_name cho 3 tệp train_data, validation_data và test_data
train_data['brand_name'] = clean_cat(train_data['brand_name'])
validation_data['brand_name'] = clean_cat(validation_data['brand_name'])
test_data['brand_name'] = clean_cat(test_data['brand_name'])

#**Vectorization**

In [ ]:
#
from sklearn.feature_extraction.text import CountVectorizer

countvectorizer=CountVectorizer().fit(train_data['sub_category1'])                 
bow_cat1_train=countvectorizer.transform(train_data['sub_category1'])
bow_cat1_val=countvectorizer.transform(validation_data['sub_category1'])
bow_cat1_test=countvectorizer.transform(test_data['sub_category1'])
print("Shape of sub_category1 features:")
print(bow_cat1_train.shape)
print(bow_cat1_val.shape)
print(bow_cat1_test.shape)
print("Some Features are:")
print(countvectorizer.get_feature_names())

In [ ]:
countvectorizer=CountVectorizer().fit(train_data['sub_category2'])   
bow_cat2_train=countvectorizer.transform(train_data['sub_category2'])
bow_cat2_val=countvectorizer.transform(validation_data['sub_category2'])
bow_cat2_test=countvectorizer.transform(test_data['sub_category2'])
print("Shape of sub_category2 features:")
print(bow_cat2_train.shape)
print(bow_cat2_val.shape)
print(bow_cat2_test.shape)
print("Some Features are: ")
print(countvectorizer.get_feature_names()[50:60])

In [ ]:
countvectorizer=CountVectorizer().fit(train_data['sub_category3'])   
bow_cat3_train=countvectorizer.transform(train_data['sub_category3'])
bow_cat3_val=countvectorizer.transform(validation_data['sub_category3'])
bow_cat3_test=countvectorizer.transform(test_data['sub_category3'])
("Shape of sub_category3 features:")
print(bow_cat3_train.shape)
print(bow_cat3_val.shape)
print(bow_cat3_test.shape)
print("Some Features are: ")
print(countvectorizer.get_feature_names()[200:210])

In [ ]:
countvectorizer=CountVectorizer().fit(train_data['brand_name'])  
bow_brand_train=countvectorizer.transform(train_data['brand_name'])
bow_brand_val=countvectorizer.transform(validation_data['brand_name'])
bow_brand_test=countvectorizer.transform(test_data['brand_name'])
("Shape of brand_name features:")
print(bow_brand_train.shape)
print(bow_brand_val.shape)
print(bow_brand_test.shape)
print("Some Features are: ")
print(countvectorizer.get_feature_names()[35:45])

In [ ]:
countvectorizer=CountVectorizer(min_df=10).fit(train_data['name'])  
bow_name_train=countvectorizer.transform(train_data['name'])
bow_name_val=countvectorizer.transform(validation_data['name'])
bow_name_test=countvectorizer.transform(test_data['name'])
print("After Vectorization of name feature: ")
print(bow_name_train.shape)
print(bow_name_val.shape)
print(bow_name_test.shape)
print("Some Features are: ")
print(countvectorizer.get_feature_names()[210:220])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvectorizer=TfidfVectorizer(ngram_range=(1,2),min_df=10,max_features=5000).fit(train_data['item_description']) 
tfidf_description_train=tfidfvectorizer.transform(train_data['item_description'])
tfidf_description_val=tfidfvectorizer.transform(validation_data['item_description'])
tfidf_description_test=tfidfvectorizer.transform(test_data['item_description'])
print("After Vectorization of item description feature: ")
print(tfidf_description_train.shape)
print(tfidf_description_val.shape)
print(tfidf_description_test.shape)
print("Some Features are: ")
print(tfidfvectorizer.get_feature_names()[222:234])

In [ ]:
from scipy.sparse import csr_matrix

features_train = csr_matrix(pd.get_dummies(train_data[['item_condition_id', 'shipping']],sparse=True).values)
features_val = csr_matrix(pd.get_dummies(validation_data[['item_condition_id', 'shipping']],sparse=True).values)
features_test = csr_matrix(pd.get_dummies(test_data[['item_condition_id', 'shipping']],sparse=True).values)
print(features_train.shape)
print(features_val.shape)
print(features_test.shape)

In [ ]:
from scipy.sparse import hstack
X_train=hstack((bow_cat1_train,bow_cat2_train,bow_cat3_train,bow_brand_train,bow_name_train,tfidf_description_train,features_train)).tocsr()
X_val=hstack((bow_cat1_val,bow_cat2_val,bow_cat3_val,bow_brand_val,bow_name_val,tfidf_description_val,features_val)).tocsr()
X_test=hstack((bow_cat1_test,bow_cat2_test,bow_cat3_test,bow_brand_test,bow_name_test,tfidf_description_test,features_test)).tocsr()
print("Shape of train data: ",X_train.shape) 
print("Shape of cv data: ",X_val.shape)   
print("Shape of test data: ",X_test.shape)   

#**Model 1:Linear regression**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

linearregression=LinearRegression(normalize=True)
linearregression.fit(X_train,train_data['log_prices'])  
ytrain_predict=linearregression.predict(X_train)
yval_predict=linearregression.predict(X_val)
train_error=np.sqrt(mean_squared_log_error(train_data['log_prices'],ytrain_predict))
val_error=np.sqrt(mean_squared_log_error(validation_data['log_prices'],yval_predict))
print(" Linear Regression RMSLE on train is {} RMSLE on cv is {}".format(train_error,val_error))



In [ ]:
yval_linear=linearregression.predict(X_val)
ytest_linear=linearregression.predict(X_test)

In [ ]:
submission_data = pd.read_csv('sample_submission_stg2.csv')
submission_data.head(5)

In [ ]:
submission_data.shape

In [ ]:
submission_data.loc[:, 'price'] = np.expm1(ytest_linear)

In [ ]:
submission_data.head(5)

In [ ]:
submission_data.to_csv('submission.csv', index=False)

In [ ]:
# train_data = train[['name', 'price', 'item_condition_id', 'brand_name', 'shipping', 'item_description', 'cat_1', 'cat_2', 'cat_3']]
# test = test[['name', 'item_condition_id', 'brand_name', 'shipping', 'item_description', 'cat_1', 'cat_2', 'cat_3']]